In [1]:
# keras - neural networks framework
import keras
from keras import *
from keras.models import Sequential
from keras.layers import *

# pandas and numpy to format the data
import pandas as pd
import numpy as np

# scikitlearn to split and normalize data
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler

# other packages
import re
import warnings

warnings.filterwarnings('ignore')

/home/rafael/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# functions to fix data types using apply from pandas

def only_nbr(x):
    return float(re.sub("[^0-9\.]", "", str(x)))

def to_float(x):
    return float(re.sub("\,", ".", str(x)))

def to_bin(x):
    if(x):
        return 1
    else:
        return 0

In [4]:
#reading data

x = pd.read_csv('dataset_X_y.csv') #mudar aqui

In [5]:
for i in range(10,38):
    x.iloc[:,i] = x.iloc[:,i].apply(to_bin)

# dropping non-numeric and unusable columns
x = x.drop(columns=['ano_censo', 'cod_escola', 'cod_municipio', 'municipio', 'regiao', 'unidade_federativa', 'Rede'])
x['NotaProvaBrasil_MT_current_year'] = x['NotaProvaBrasil_MT_current_year'].apply(only_nbr)
x['NotaProvaBrasil_LP_current_year'] = x['NotaProvaBrasil_LP_current_year'].apply(only_nbr)

# creating ground truth
y = x['Ideb_next_2_years']

# excluding from inputs
x = x.drop(columns=['Ideb_next_2_years'])
input_dim = len(x.columns)
x_df = x
# normalizing data 0-1
scaler = MinMaxScaler()
scaler.fit(x)
x = scaler.transform(x)

# split train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75)

In [6]:
# models
def create_model1():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=input_dim))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    adam = optimizers.Adam(lr = 0.001)
    model.compile(adam, loss='mean_squared_error')
    return model

def create_model2():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=input_dim))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    adam = optimizers.Adam(lr = 0.001)
    model.compile(adam, loss='mean_squared_error')
    return model

In [7]:
# it's not k-fold cross validation, but helps to understand how is the loss in more than one case

k = 5
loss1 = []
loss2 = []
for i in range(1,k+1):
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=i)
    
    model1 = create_model1()
    es = callbacks.EarlyStopping(monitor='val_loss', patience = 10)
    model1.fit(x_train, y_train, batch_size=32, epochs=200, callbacks=[es], validation_split=0.25)
    loss1.append(model1.evaluate(x_test, y_test))
    
    model2 = create_model2()
    es = callbacks.EarlyStopping(monitor='val_loss', patience = 10)
    model2.fit(x_train, y_train, batch_size=32, epochs=200, callbacks=[es], validation_split=0.25)
    loss2.append(model2.evaluate(x_test, y_test))

Train on 46146 samples, validate on 15383 samples
Epoch 1/200
46146/46146 [==============================] - 1s 29us/step - loss: 0.5121 - val_loss: 0.2959
Epoch 2/200
46146/46146 [==============================] - 1s 21us/step - loss: 0.2824 - val_loss: 0.3075
Epoch 3/200
46146/46146 [==============================] - 1s 22us/step - loss: 0.2816 - val_loss: 0.2865
Epoch 4/200
46146/46146 [==============================] - 1s 22us/step - loss: 0.2807 - val_loss: 0.2871
Epoch 5/200
46146/46146 [==============================] - 1s 21us/step - loss: 0.2780 - val_loss: 0.2898
Epoch 6/200
46146/46146 [==============================] - 1s 21us/step - loss: 0.2785 - val_loss: 0.2849
Epoch 7/200
46146/46146 [==============================] - 1s 24us/step - loss: 0.2776 - val_loss: 0.2813
Epoch 8/200
46146/46146 [==============================] - 1s 23us/step - loss: 0.2754 - val_loss: 0.2784
Epoch 9/200
46146/46146 [==============================] - 1s 22us/step - loss: 0.2757 - val_loss: 0.2

46146/46146 [==============================] - 1s 22us/step - loss: 0.2701 - val_loss: 0.2731
Epoch 23/200
46146/46146 [==============================] - 1s 23us/step - loss: 0.2693 - val_loss: 0.2748
Epoch 24/200
46146/46146 [==============================] - 1s 23us/step - loss: 0.2695 - val_loss: 0.2789
Epoch 25/200
46146/46146 [==============================] - 1s 22us/step - loss: 0.2693 - val_loss: 0.2771
Epoch 26/200
46146/46146 [==============================] - 1s 22us/step - loss: 0.2685 - val_loss: 0.2755
Epoch 27/200
46146/46146 [==============================] - 1s 23us/step - loss: 0.2682 - val_loss: 0.2749
Epoch 28/200
46146/46146 [==============================] - 1s 23us/step - loss: 0.2673 - val_loss: 0.2774
Epoch 29/200
46146/46146 [==============================] - 1s 23us/step - loss: 0.2671 - val_loss: 0.2735
Epoch 30/200
46146/46146 [==============================] - 1s 23us/step - loss: 0.2673 - val_loss: 0.2786
Epoch 31/200
46146/46146 [========================

Epoch 18/200
46146/46146 [==============================] - 1s 23us/step - loss: 0.2684 - val_loss: 0.3106
Epoch 19/200
46146/46146 [==============================] - 1s 26us/step - loss: 0.2678 - val_loss: 0.2870
Epoch 20/200
46146/46146 [==============================] - 1s 22us/step - loss: 0.2678 - val_loss: 0.2844
Epoch 21/200
46146/46146 [==============================] - 1s 24us/step - loss: 0.2669 - val_loss: 0.2747
Epoch 22/200
46146/46146 [==============================] - 1s 25us/step - loss: 0.2669 - val_loss: 0.2741
Epoch 23/200
46146/46146 [==============================] - 1s 25us/step - loss: 0.2664 - val_loss: 0.2778
Epoch 24/200
46146/46146 [==============================] - 1s 22us/step - loss: 0.2658 - val_loss: 0.2888
Epoch 25/200
46146/46146 [==============================] - 1s 22us/step - loss: 0.2656 - val_loss: 0.2755
Epoch 26/200
46146/46146 [==============================] - 1s 25us/step - loss: 0.2650 - val_loss: 0.3368
Epoch 27/200
46146/46146 [===========

46146/46146 [==============================] - 1s 25us/step - loss: 0.2760 - val_loss: 0.2871
Epoch 8/200
46146/46146 [==============================] - 1s 25us/step - loss: 0.2775 - val_loss: 0.2884
Epoch 9/200
46146/46146 [==============================] - 1s 25us/step - loss: 0.2744 - val_loss: 0.2830
Epoch 10/200
46146/46146 [==============================] - 1s 25us/step - loss: 0.2745 - val_loss: 0.2763
Epoch 11/200
46146/46146 [==============================] - 1s 25us/step - loss: 0.2729 - val_loss: 0.2775
Epoch 12/200
46146/46146 [==============================] - 1s 25us/step - loss: 0.2739 - val_loss: 0.2987
Epoch 13/200
46146/46146 [==============================] - 1s 25us/step - loss: 0.2709 - val_loss: 0.2894
Epoch 14/200
46146/46146 [==============================] - 1s 26us/step - loss: 0.2706 - val_loss: 0.2795
Epoch 15/200
46146/46146 [==============================] - 1s 26us/step - loss: 0.2702 - val_loss: 0.2807
Epoch 16/200
46146/46146 [==========================

In [8]:
# mean MSE
print(np.mean(loss1))
print(np.mean(loss2))

0.28229642398615457
0.2743664767153957


In [9]:
# Analyzing growth and decay to create a confusion matrix

new_scaler = MinMaxScaler()
new_scaler.fit([[scaler.data_min_[-3]], [scaler.data_max_[-3]]])

model1_n = new_scaler.transform(model1.predict(x_test)[:,:])
model2_n = new_scaler.transform(model2.predict(x_test)[:,:])
y_test_n = new_scaler.transform([np.array(y_test)])
y_test_n = np.transpose(y_test_n)
x_test_n = [x_test[:,-3]]
x_test_n = np.transpose(x_test_n)

# predito - atual
d1 = model1_n - x_test_n
d2 = model2_n - x_test_n

# real - atual
d3 = y_test_n - x_test_n

In [10]:
# generating confusion matrix

kind1 = []
kind2 = []
a = -1
b = -1
for i, j, k in zip(d1,d2,d3):
    if(i >= 0 and k > 0): ## true positive
        a = 1
    elif(i >= 0 and k < 0): ## false positive
        a = 2
    elif(i < 0 and k > 0): ## false negative
        a = 3
    else: ## true negative
        a = 4
    
    if(j >= 0 and k > 0): ## true positive
        b = 1
    elif(j > 0 and k < 0): ## false positive
        b = 2
    elif(j < 0 and k > 0): ## false negative
        b = 3
    else: ## true negative
        b = 4
    kind1.append(a)
    kind2.append(b)

In [11]:
confusion_matrix = [kind1.count(1), kind1.count(2), kind1.count(3), kind1.count(4)]
print(confusion_matrix)
confusion_matrix = [kind2.count(1), kind2.count(2), kind2.count(3), kind2.count(4)]
print(confusion_matrix)

[9953, 4281, 1896, 4380]
[10050, 4343, 1799, 4318]


In [12]:
w = model2.layers[0].get_weights()[0].sum(axis=1)

In [13]:
# verifying weights of the first layer

dict = {}
for i in range(56):
    dict[x_df.columns[i]] = w[i]

for i in dict:
    print(i)
    print(dict[i])
    print('----------------------------')

pib
-5.068445
----------------------------
pop
-2.6512885
----------------------------
num_estudantes_ensino_fund_anos_finais
-5.024829
----------------------------
num_professores_em_regencia_fund_af
-1.0396985
----------------------------
in_agua_filtrada
-2.6972737
----------------------------
in_agua_inexistente
0.92397934
----------------------------
in_energia_rede_publica
-1.2199844
----------------------------
in_energia_inexistente
-1.4226849
----------------------------
in_esgoto_rede_publica
-1.4637947
----------------------------
in_esgoto_inexistente
-0.5608607
----------------------------
in_sala_professor
-2.369531
----------------------------
in_laboratorio_informatica
-2.3656266
----------------------------
in_laboratorio_ciencias
-1.0382903
----------------------------
in_sala_atendimento_especial
-1.0101061
----------------------------
in_quadra_esportes
0.14796466
----------------------------
in_cozinha
-1.3042468
----------------------------
in_biblioteca
-1.722273